<https://github.com/PolymathicAI/xVal>


In [1]:
import os

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.95"

In [2]:
from datetime import datetime as dt
from typing import Union

import icecream
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import orbax
import orbax.checkpoint
import pandas as pd
from flax.struct import dataclass
from flax.training import orbax_utils, train_state
from icecream import ic
from jax import random
from jax.tree_util import tree_flatten
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange
from transformers import BertTokenizerFast, FlaxBertModel

import hephaestus as hp
import hephaestus.models.time_series_decoder_training as tsd
import hashlib
import random as py_random
import nltk
from nltk.corpus import words

# Predefined list of words


# Download the words corpus if not already downloaded
nltk.download("words")

icecream.install()
ic_disable = False  # Global variable to disable ic
if ic_disable:
    ic.disable()
ic.configureOutput(includeContext=True, contextAbsPath=True)
pd.options.mode.copy_on_write = True

[nltk_data] Downloading package words to
[nltk_data]     /Users/kailukowiak/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
word_list = words.words()

In [4]:
# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
model = FlaxBertModel.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# Get the embeddings matrix
embeddings = model.params["embeddings"]["word_embeddings"]["embedding"]

# Now you can access specific embeddings like this:
# For example, to get embeddings for tokens 23, 293, and 993:
selected_embeddings = jnp.take(embeddings, jnp.array([23, 293, 993]), axis=0)

# If you want to get embeddings for specific words:
words = ["hello", "world", "example"]
tokens = tokenizer.convert_tokens_to_ids(words)
word_embeddings = jnp.take(embeddings, jnp.array(tokens), axis=0)
word_embeddings.shape

Some weights of FlaxBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: {('pooler', 'dense', 'kernel'), ('pooler', 'dense', 'bias')}
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(3, 768)

In [5]:
data_path = "data/Untouched Data - 1 Oct 24"
csvs = os.listdir(data_path)
dfs = []
for csv in csvs:
    df = pd.read_csv(os.path.join(data_path, csv))
    dfs.append(df)
df = pd.concat(dfs)
df.head()

,id,created_at,updated_at,deleted_at,date,in_amount,in_token,in_from,out_amount,out_token,...,exchange,type_id,transaction_hash,wallet_id,chain,usd_equivalent,user_id,additional_transaction_data,deleted,status
0,004496fb-2a79-4ad7-84c1-300b16bafabb,2024-10-01 04:58:33.77301+00,2024-10-01 04:58:33.77301+00,NaN,2022-12-30T23:23:35-07:00,8.234850e-01,WETH,0xab224617a340a91053b57e0189d137beadf32b5d,8.234850e-01,WETH,...,NaN,transfer,0x4e5add7e23727299096694f207cb45f7d5d8b5dc0877...,86c133c0-9747-47af-9d8e-6ea7aaad7951,ethereum,NaN,169885e1-baef-4163-aefe-47b664a1d375,NaN,False,active
1,0077a1cc-2f26-467c-883f-47332c86b949,2024-10-01 04:58:35.14311+00,2024-10-01 04:58:35.14311+00,NaN,2022-10-17T15:19:11-06:00,9.385498e+05,SCP,0xb6037a4dfe560f56206b45f0589b23b2380dbf77,9.385498e+05,SCP,...,NaN,transfer,0xb22f03fab1212556941b1b580fe98adb607fc3d55dcc...,86c133c0-9747-47af-9d8e-6ea7aaad7951,ethereum,NaN,169885e1-baef-4163-aefe-47b664a1d375,NaN,False,active
2,007e86e3-8f88-433c-be0e-0e68293552ef,2024-10-01 04:58:38.341808+00,2024-10-01 04:58:38.341808+00,NaN,2022-05-08T20:35:46-06:00,8.952095e+07,QUEENDOM,0x619e194aff11a10a5348cf32fd2240e534ab8f90,8.952095e+07,QUEENDOM,...,NaN,transfer,0x9f7b64f3c07d9f495beb080838d9db00d4f477eded97...,86c133c0-9747-47af-9d8e-6ea7aaad7951,ethereum,NaN,169885e1-baef-4163-aefe-47b664a1d375,NaN,False,active
3,00aa6bd3-33b2-40df-80bc-846b2af0fdb3,2024-10-01 04:58:38.341808+00,2024-10-01 04:58:38.341808+00,NaN,2022-05-06T07:15:49-06:00,1.372072e+07,GFMI,0xb71deac7964755d23b316c0044afbbc5eff2b09c,1.372072e+07,GFMI,...,NaN,transfer,0x6ffb676605fda9721a8c9d9d254a00f70561f98b2ae6...,86c133c0-9747-47af-9d8e-6ea7aaad7951,ethereum,NaN,169885e1-baef-4163-aefe-47b664a1d375,NaN,False,active
4,011dc7b4-a869-41b1-a2f7-e1e138591a7b,2024-10-01 04:58:35.14311+00,2024-10-01 04:58:35.14311+00,NaN,2022-10-17T14:00:23-06:00,3.000000e+00,ETH,0xab224617a340a91053b57e0189d137beadf32b5d,3.000000e+00,ETH,...,NaN,transfer,0x944fb012efe2aa325f246794523d7f98db94e9fe685f...,86c133c0-9747-47af-9d8e-6ea7aaad7951,ethereum,NaN,169885e1-baef-4163-aefe-47b664a1d375,NaN,False,active


In [6]:
datetime_cols = ["created_at", "updated_at", "date"]


def enrich_datetimes(df: pd.DataFrame, col: str):
    df[col] = pd.to_datetime(df[col], utc=True)

    df[f"{col}_year"] = df[col].dt.year
    df[f"{col}_month_sin"] = np.sin(2 * np.pi * df[col].dt.month / 12)
    df[f"{col}_month_cos"] = np.cos(2 * np.pi * df[col].dt.month / 12)
    df[f"{col}_day_sin"] = np.sin(2 * np.pi * df[col].dt.day / 31)
    df[f"{col}_day_cos"] = np.cos(2 * np.pi * df[col].dt.day / 31)
    df[f"{col}_hour_sin"] = np.sin(2 * np.pi * df[col].dt.hour / 24)
    df[f"{col}_hour_cos"] = np.cos(2 * np.pi * df[col].dt.hour / 24)
    df[f"{col}_minute_sin"] = np.sin(2 * np.pi * df[col].dt.minute / 60)
    df[f"{col}_minute_cos"] = np.cos(2 * np.pi * df[col].dt.minute / 60)
    df[f"{col}_second_sin"] = np.sin(2 * np.pi * df[col].dt.second / 60)
    df[f"{col}_second_cos"] = np.cos(2 * np.pi * df[col].dt.second / 60)

    return df.drop(columns=[col])


for col in datetime_cols:
    df = enrich_datetimes(df, col)

In [7]:
# Use the words corpus from nltk
# words_list = words.words()


def hash_to_words(crypto_hash: Union[str, np.nan]) -> Union[str, np.nan]:
    if crypto_hash is np.nan:
        return np.nan
    # Create a hash of the input
    hash_object = hashlib.sha256(crypto_hash.encode())
    hash_digest = hash_object.hexdigest()

    # Use the hash to select three words from the list
    py_random.seed(hash_digest)
    selected_words = py_random.sample(word_list, 3)

    return "hash " + " ".join(selected_words)


# Example usage
crypto_hash = "269CiLPaFK55QqiVbsJupN6BSPUHQ3x7kN6iHPhSMV2NDwHM2EhnwQ6hE6FEvbix6AVN2PLUMQyyhrKr2y514dRB"
print(hash_to_words(crypto_hash))

hash rabies placophoran isagogically


In [8]:
1 / 0

ZeroDivisionError: division by zero

In [9]:
hash_columns = [
    "id",
    "in_from",
    "out_to",
    "fee_paid_by",
    "transaction_hash",
    "wallet_id",
    "user_id",
]
for col in hash_columns:
    df[col] = df[col].apply(hash_to_words)
df.head()

,id,deleted_at,in_amount,in_token,in_from,out_amount,out_token,out_to,fee_amount,fee_paid_by,...,date_month_sin,date_month_cos,date_day_sin,date_day_cos,date_hour_sin,date_hour_cos,date_minute_sin,date_minute_cos,date_second_sin,date_second_cos
0,hash Ganodonta competently outpension,NaN,8.234850e-01,WETH,hash antepectus paganism unforeknowable,8.234850e-01,WETH,hash menthenol Babylonish deviscerate,0.003935,hash antepectus paganism unforeknowable,...,-2.449294e-16,1.000000,-2.449294e-16,1.000000,1.000000,6.123234e-17,0.669131,-7.431448e-01,-0.500000,-0.866025
1,hash unitage grandchild carless,NaN,9.385498e+05,SCP,hash metatoluidine transducer prefragrance,9.385498e+05,SCP,hash antepectus paganism unforeknowable,NaN,NaN,...,-8.660254e-01,0.500000,-2.993631e-01,-0.954139,-0.707107,7.071068e-01,0.913545,-4.067366e-01,0.913545,0.406737
2,hash boonk floweret unimbued,NaN,8.952095e+07,QUEENDOM,hash coulure teamster prehistoric,8.952095e+07,QUEENDOM,hash antepectus paganism unforeknowable,NaN,NaN,...,5.000000e-01,-0.866025,9.680771e-01,-0.250653,0.500000,8.660254e-01,-0.500000,-8.660254e-01,-0.994522,0.104528
3,hash emersion Laudist surely,NaN,1.372072e+07,GFMI,hash bipunctate suctorial buoyantly,1.372072e+07,GFMI,hash antepectus paganism unforeknowable,NaN,NaN,...,5.000000e-01,-0.866025,9.377521e-01,0.347305,-0.258819,-9.659258e-01,1.000000,2.832769e-16,-0.913545,0.406737
4,hash addleheaded valvular Mississippian,NaN,3.000000e+00,ETH,hash antepectus paganism unforeknowable,3.000000e+00,ETH,hash titanium splenomegaly placarder,0.004935,hash antepectus paganism unforeknowable,...,-8.660254e-01,0.500000,-2.993631e-01,-0.954139,-0.866025,5.000000e-01,0.000000,1.000000e+00,0.669131,-0.743145


In [10]:
df.dtypes

id                              object
deleted_at                     float64
in_amount                      float64
in_token                        object
in_from                         object
out_amount                     float64
out_token                       object
out_to                          object
fee_amount                     float64
fee_paid_by                     object
exchange                       float64
type_id                         object
transaction_hash                object
wallet_id                       object
chain                           object
usd_equivalent                 float64
user_id                         object
additional_transaction_data    float64
deleted                           bool
status                          object
created_at_year                  int32
created_at_month_sin           float64
created_at_month_cos           float64
created_at_day_sin             float64
created_at_day_cos             float64
created_at_hour_sin      

In [11]:
obj_cols = df.select_dtypes(include=["object"]).columns
num_cols = df.select_dtypes(include=["number"]).columns
df[obj_cols] = df[obj_cols].fillna("missing")
df[num_cols] = df[num_cols].fillna(0)
df.head()

,id,deleted_at,in_amount,in_token,in_from,out_amount,out_token,out_to,fee_amount,fee_paid_by,...,date_month_sin,date_month_cos,date_day_sin,date_day_cos,date_hour_sin,date_hour_cos,date_minute_sin,date_minute_cos,date_second_sin,date_second_cos
0,hash Ganodonta competently outpension,0.0,8.234850e-01,WETH,hash antepectus paganism unforeknowable,8.234850e-01,WETH,hash menthenol Babylonish deviscerate,0.003935,hash antepectus paganism unforeknowable,...,-2.449294e-16,1.000000,-2.449294e-16,1.000000,1.000000,6.123234e-17,0.669131,-7.431448e-01,-0.500000,-0.866025
1,hash unitage grandchild carless,0.0,9.385498e+05,SCP,hash metatoluidine transducer prefragrance,9.385498e+05,SCP,hash antepectus paganism unforeknowable,0.000000,missing,...,-8.660254e-01,0.500000,-2.993631e-01,-0.954139,-0.707107,7.071068e-01,0.913545,-4.067366e-01,0.913545,0.406737
2,hash boonk floweret unimbued,0.0,8.952095e+07,QUEENDOM,hash coulure teamster prehistoric,8.952095e+07,QUEENDOM,hash antepectus paganism unforeknowable,0.000000,missing,...,5.000000e-01,-0.866025,9.680771e-01,-0.250653,0.500000,8.660254e-01,-0.500000,-8.660254e-01,-0.994522,0.104528
3,hash emersion Laudist surely,0.0,1.372072e+07,GFMI,hash bipunctate suctorial buoyantly,1.372072e+07,GFMI,hash antepectus paganism unforeknowable,0.000000,missing,...,5.000000e-01,-0.866025,9.377521e-01,0.347305,-0.258819,-9.659258e-01,1.000000,2.832769e-16,-0.913545,0.406737
4,hash addleheaded valvular Mississippian,0.0,3.000000e+00,ETH,hash antepectus paganism unforeknowable,3.000000e+00,ETH,hash titanium splenomegaly placarder,0.004935,hash antepectus paganism unforeknowable,...,-8.660254e-01,0.500000,-2.993631e-01,-0.954139,-0.866025,5.000000e-01,0.000000,1.000000e+00,0.669131,-0.743145


In [12]:
df.dtypes

id                              object
deleted_at                     float64
in_amount                      float64
in_token                        object
in_from                         object
out_amount                     float64
out_token                       object
out_to                          object
fee_amount                     float64
fee_paid_by                     object
exchange                       float64
type_id                         object
transaction_hash                object
wallet_id                       object
chain                           object
usd_equivalent                 float64
user_id                         object
additional_transaction_data    float64
deleted                           bool
status                          object
created_at_year                  int32
created_at_month_sin           float64
created_at_month_cos           float64
created_at_day_sin             float64
created_at_day_cos             float64
created_at_hour_sin      

In [14]:
df["idx"] = df.index // 10
df.head(20)

,id,deleted_at,in_amount,in_token,in_from,out_amount,out_token,out_to,fee_amount,fee_paid_by,...,date_month_cos,date_day_sin,date_day_cos,date_hour_sin,date_hour_cos,date_minute_sin,date_minute_cos,date_second_sin,date_second_cos,idx
0,hash Ganodonta competently outpension,0.0,8.234850e-01,WETH,hash antepectus paganism unforeknowable,8.234850e-01,WETH,hash menthenol Babylonish deviscerate,0.003935,hash antepectus paganism unforeknowable,...,1.000000e+00,-2.449294e-16,1.000000,1.000000,6.123234e-17,6.691306e-01,-7.431448e-01,-5.000000e-01,-8.660254e-01,0
1,hash unitage grandchild carless,0.0,9.385498e+05,SCP,hash metatoluidine transducer prefragrance,9.385498e+05,SCP,hash antepectus paganism unforeknowable,0.000000,missing,...,5.000000e-01,-2.993631e-01,-0.954139,-0.707107,7.071068e-01,9.135455e-01,-4.067366e-01,9.135455e-01,4.067366e-01,0
2,hash boonk floweret unimbued,0.0,8.952095e+07,QUEENDOM,hash coulure teamster prehistoric,8.952095e+07,QUEENDOM,hash antepectus paganism unforeknowable,0.000000,missing,...,-8.660254e-01,9.680771e-01,-0.250653,0.500000,8.660254e-01,-5.000000e-01,-8.660254e-01,-9.945219e-01,1.045285e-01,0
3,hash emersion Laudist surely,0.0,1.372072e+07,GFMI,hash bipunctate suctorial buoyantly,1.372072e+07,GFMI,hash antepectus paganism unforeknowable,0.000000,missing,...,-8.660254e-01,9.377521e-01,0.347305,-0.258819,-9.659258e-01,1.000000e+00,2.832769e-16,-9.135455e-01,4.067366e-01,0
4,hash addleheaded valvular Mississippian,0.0,3.000000e+00,ETH,hash antepectus paganism unforeknowable,3.000000e+00,ETH,hash titanium splenomegaly placarder,0.004935,hash antepectus paganism unforeknowable,...,5.000000e-01,-2.993631e-01,-0.954139,-0.866025,5.000000e-01,0.000000e+00,1.000000e+00,6.691306e-01,-7.431448e-01,0
5,hash patness maya shut,0.0,1.606460e-01,ETH,hash titanium splenomegaly placarder,1.606460e-01,ETH,hash antepectus paganism unforeknowable,0.000000,missing,...,8.660254e-01,8.978045e-01,-0.440394,0.965926,-2.588190e-01,9.135455e-01,-4.067366e-01,9.135455e-01,4.067366e-01,0
6,hash Distomatidae beefy prejournalistic,0.0,5.049639e+00,ETH,hash titanium splenomegaly placarder,5.049639e+00,ETH,hash antepectus paganism unforeknowable,0.000000,missing,...,-5.000000e-01,5.712682e-01,0.820763,-0.258819,-9.659258e-01,9.781476e-01,2.079117e-01,1.045285e-01,-9.945219e-01,0
7,hash swifty parsonity pritchel,0.0,1.469171e+05,$TDAO,hash antepectus paganism unforeknowable,1.469171e+05,$TDAO,hash verminal strome contra,0.005567,hash antepectus paganism unforeknowable,...,-5.000000e-01,-1.011683e-01,-0.994869,0.965926,-2.588190e-01,-5.877853e-01,-8.090170e-01,5.665539e-16,-1.000000e+00,0
8,hash discomfiture Irishry pronominal,0.0,8.000000e-02,ETH,hash antepectus paganism unforeknowable,8.000000e-02,ETH,hash titanium splenomegaly placarder,0.003000,hash antepectus paganism unforeknowable,...,8.660254e-01,-1.011683e-01,-0.994869,0.707107,-7.071068e-01,8.090170e-01,5.877853e-01,6.691306e-01,-7.431448e-01,0
9,hash plugtray cader walth,0.0,6.695540e-01,ETH,hash titanium splenomegaly placarder,6.695540e-01,ETH,hash antepectus paganism unforeknowable,0.000000,missing,...,-8.660254e-01,-7.907757e-01,-0.612106,-0.965926,-2.588190e-01,2.079117e-01,-9.781476e-01,1.000000e+00,2.832769e-16,0


In [15]:
# Get train test split at 80/20
time_series_config = hp.TimeSeriesConfig.generate(df=df)
train_idx = int(df.idx.max() * 0.8)
train_df = df.loc[df.idx < train_idx].copy()
test_df = df.loc[df.idx >= train_idx].copy()
# del df
train_ds = hp.TimeSeriesDS(train_df, time_series_config)
test_ds = hp.TimeSeriesDS(test_df, time_series_config)
len(train_ds), len(test_ds)

(601, 152)

In [16]:
def make_batch(ds: hp.TimeSeriesDS, start: int, length: int):
    numeric = []
    categorical = []
    for i in range(start, length + start):
        numeric.append(ds[i][0])
        categorical.append(ds[i][1])
    # print index of None values
    return {"numeric": jnp.array(numeric), "categorical": jnp.array(categorical)}


batch = make_batch(train_ds, 0, 4)
# batch

In [17]:
multiplier = 4
time_series_regressor = hp.TimeSeriesDecoder(
    time_series_config, d_model=512, n_heads=8 * multiplier
)

In [18]:
key = random.PRNGKey(0)
init_key, dropout_key = random.split(key)
vars = time_series_regressor.init(
    {"params": init_key, "dropout": dropout_key},
    batch["numeric"],
    categorical_inputs=batch["categorical"].astype(jnp.int32),
    deterministic=False,
    causal_mask=False,
)
dropout_key, original_dropout_key = random.split(dropout_key)

ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:444 in process_numeric()
    numeric_embedding.shape: (512,)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:447 in process_numeric()
    numeric_embedding.shape: (4, 40, 100, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:455 in process_numeric()
    numeric_embedding.shape: (4, 40, 100, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:519 in combine_inputs()
    numeric.value_embeddings.shape: (4, 40, 100, 512)
    categorical.value_embeddings.shape: (4, 12, 100, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:617 in __call__()
    combined_inputs.value_embeddings.shape: (4, 52, 100, 512)
    combined_inputs.column_embeddings.shape: (4, 52, 100, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:314 in __call__()
    "Transformer Block": 'Transformer Block'
   

In [19]:
x = time_series_regressor.apply(
    vars,
    batch["numeric"],
    batch["categorical"].astype(jnp.int32),
    deterministic=False,
    rngs={"dropout": dropout_key},
)
print(x.get("numeric_out").shape)
# Check if categorical input is None and print None or it's shape
print(x.get("categorical_out").shape if x.get("categorical_out") is not None else None)

ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:444 in process_numeric()
    numeric_embedding.shape: (512,)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:447 in process_numeric()
    numeric_embedding.shape: (4, 40, 100, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:455 in process_numeric()
    numeric_embedding.shape: (4, 40, 100, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:519 in combine_inputs()
    numeric.value_embeddings.shape: (4, 40, 100, 512)
    categorical.value_embeddings.shape: (4, 12, 100, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:617 in __call__()
    combined_inputs.value_embeddings.shape: (4, 52, 100, 512)
    combined_inputs.column_embeddings.shape: (4, 52, 100, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:314 in __call__()
    "Transformer Block": 'Transformer Block'
   

(4, 40, 100)
(4, 12, 100, 45906)


In [20]:
x["numeric_out"]

Array([[[ 1.1072633 ,  0.37954804, -0.25939   , ...,  0.09006238,
          0.26905265, -0.14123808],
        [ 0.9160707 ,  0.73012656,  0.30686626, ...,  0.2008718 ,
          0.16510844, -0.04005447],
        [ 0.7175883 ,  0.09832948,  0.21641402, ..., -0.09768087,
         -0.16773844,  0.0355442 ],
        ...,
        [ 0.08242998, -0.4711998 , -0.298275  , ..., -0.62759703,
         -0.72137904, -0.38742638],
        [-0.21430117,  0.23132947,  0.06126913, ..., -0.49833882,
         -0.25192603, -0.46959668],
        [ 1.1101143 , -0.26705888,  0.5200216 , ...,  0.18262792,
          0.3771929 ,  0.40181255]],

       [[-0.13173485,  0.33314046, -0.35449967, ...,  0.3839442 ,
          0.30709544,  0.31475705],
        [ 0.44977397, -0.2782897 ,  0.35140356, ...,  0.43094906,
          0.2887856 ,  0.02127253],
        [ 0.13765252, -0.08726341,  0.14211994, ...,  0.78127575,
          0.13684186,  0.1589626 ],
        ...,
        [-0.1901573 , -0.08559008,  0.22582434, ..., -

In [21]:
def calculate_memory_footprint(params):
    """Calculate total memory footprint of JAX model parameters and total
    number of parameters."""
    total_bytes = 0
    # Flatten the parameter tree structure into a list of arrays
    flat_params, _ = tree_flatten(params)
    for param in flat_params:
        # Calculate bytes: number of elements * size of each element
        bytes_per_param = param.size * param.dtype.itemsize
        total_bytes += bytes_per_param
    return total_bytes


def count_parameters(params):
    return sum(jnp.prod(jnp.array(p.shape)) for p in jax.tree_util.tree_leaves(params))


mem = calculate_memory_footprint(vars)
total_params = count_parameters(vars)


print(f"Memory of custom: {mem / 1e6:.2f} MB with {total_params:,} parameters")

Memory of custom: 315.34 MB with 78,834,422 parameters


In [22]:
ic.disable()

In [23]:
mts_root_key = random.PRNGKey(44)
mts_main_key, ts_params_key, ts_data_key = random.split(mts_root_key, 3)

causal_mask = False


batch_size = 2

state = tsd.create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)

In [25]:
causal_mask = False

In [27]:
writer_name = "NewCryptoTransactions"

writer_time = dt.now().strftime("%Y-%m-%dT%H:%M:%S")
model_name = writer_time + writer_name
train_summary_writer = SummaryWriter("runs/" + model_name)


test_set_key = random.PRNGKey(444)

batch_size = 4
train_data_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

# train_data_loader = DataLoader(train_ds, batch_size=256 // 2, shuffle=True)
# test_data_loader = DataLoader(test_ds, batch_size=256 // 2, shuffle=True)

batch_count = 0
base_key = random.PRNGKey(42)

# Disable IC for training
max_iters = None
ic.disable()
for j in trange(1, desc=f"epochs for {train_summary_writer.log_dir}"):
    # arrs = train_data_loader()
    for i in tqdm(train_data_loader, leave=False, desc="batches"):
        # for i in trange(len(pre_train) // batch_size, leave=False):
        # for i in trange(len(pre_train) // batch_size //10, leave=False):
        # batch = make_batch(train_ds, i[0], 4)
        state, loss, base_key = tsd.train_step(
            state,
            jnp.array(i[0]),
            jnp.array(i[1]),
            base_key,
            # causal_mask=causal_mask,
        )
        if jnp.isnan(loss):
            raise ValueError("Nan Value in loss, stopping")
        batch_count += 1

        if batch_count % 1 == 0:
            train_summary_writer.add_scalar(
                "loss/loss", np.array(loss.item()), batch_count
            )
        if batch_count % 10 == 0:
            numeric_eval, categorical_eval = next(iter(test_data_loader))
            test_loss, base_key = tsd.eval_step(
                state,
                jnp.array(numeric_eval),
                jnp.array(categorical_eval),
                base_key,
                # causal_mask=causal_mask,
            )
            train_summary_writer.add_scalar(
                "loss/test_loss", np.array(test_loss.item()), batch_count
            )
            train_summary_writer.flush()
        # if batch_count > 200:
        #     break
        if not max_iters:
            continue
        else:
            if batch_count > max_iters:
                break

train_summary_writer.close()

epochs for runs/2024-10-01T21:10:38NewCryptoTransactions:   0%|          | 0/1 [00:00<?, ?it/s]

batches:   0%|          | 0/151 [00:00<?, ?it/s]

InputOffset: 1 <class 'int'>
InputOffset: 1 <class 'int'>


TypeError: softmax_cross_entropy_with_integer_labels() takes 2 positional arguments but 3 were given

In [ ]:
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()

In [ ]:
ckpt = {"model": state, "step": batch_count}


checkpoint_dir = f"checkpoints/{model_name}"
checkpoint_dir = os.path.abspath(checkpoint_dir)

# os.makedirs(checkpoint_dir, exist_ok=True)

orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
save_args = orbax_utils.save_args_from_target(ckpt)
orbax_checkpointer.save(checkpoint_dir, ckpt, save_args=save_args)

In [ ]:
# if model_name is None:
if True:
    all_checkpoints = os.listdir("checkpoints/")
    model_name = all_checkpoints
    all_checkpoints = sorted(all_checkpoints)
    model_name = all_checkpoints[-1]
    checkpoint_dir = f"checkpoints/{model_name}"
    checkpoint_dir = os.path.abspath(checkpoint_dir)

new_checkpoint = orbax_checkpointer.restore(checkpoint_dir)
new_state = tsd.create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)
new_state = new_state.replace(params=new_checkpoint["model"]["params"])

In [ ]:
numeric_inputs, categorical_inputs = train_ds[0]
numeric_inputs = jnp.array([numeric_inputs])
categorical_inputs = jnp.array([categorical_inputs])
test_results = state.apply_fn(
    {"params": state.params},
    # jnp.array(i[0]),
    # jnp.array(i[1]),
    numeric_inputs,
    categorical_inputs,
    deterministic=True,
    causal_mask=False,
)
test_results["numeric_out"].shape

In [ ]:
@dataclass
class Results:
    numeric_out: jnp.array
    categorical_out: jnp.array
    numeric_inputs: jnp.array
    categorical_inputs: jnp.array


def return_results(state, dataset, idx=0, mask_start: int = None):
    numeric_inputs, categorical_inputs = dataset[idx]
    if mask_start:
        numeric_inputs = numeric_inputs[:, :mask_start]
        categorical_inputs = categorical_inputs[:, :mask_start]
    numeric_inputs = jnp.array([numeric_inputs])
    categorical_inputs = jnp.array([categorical_inputs])
    out = state.apply_fn(
        {"params": state.params},
        # hp.mask_tensor(jnp.array([train_ds[0]]), dataset, prng_key=key),
        numeric_inputs=numeric_inputs,
        categorical_inputs=categorical_inputs,
        deterministic=True,
        causal_mask=causal_mask,
    )
    numeric_out, categorical_out = out["numeric_out"], out["categorical_out"]
    return Results(numeric_out, categorical_out, numeric_inputs, categorical_inputs)


x = return_results(state, train_ds, 0)
x.categorical_out.shape

In [ ]:
causal_mask = False
causal_mask = True


def process_results(arr: jnp.array, col_names: list, config: hp.TimeSeriesConfig):
    arr = jnp.squeeze(arr)
    if arr.ndim == 3:
        # Check if there is a logit array for example if there are 3 dims then the
        # last dim is the logit array. We need to get the argmax of the last dim
        # to get the actual values of the array and replace the logit array with the
        # actual values
        arr = jnp.argmax(arr, axis=-1)
    df = pd.DataFrame(arr.T)
    df.columns = col_names
    return df


@dataclass
class DFComparison:
    input_df: pd.DataFrame
    output_df: pd.DataFrame


def show_results_df(
    state, time_series_config, dataset, idx: int = 0, mask_start: int = None
):
    results = return_results(state, dataset, idx=idx, mask_start=mask_start)

    input_categorical = process_results(
        results.categorical_inputs,
        time_series_config.categorical_col_tokens,
        time_series_config,
    )
    input_numeric = process_results(
        results.numeric_inputs,
        time_series_config.numeric_col_tokens,
        time_series_config,
    )
    output_categorical = process_results(
        results.categorical_out,
        time_series_config.categorical_col_tokens,
        time_series_config,
    )
    output_numeric = process_results(
        results.numeric_out, time_series_config.numeric_col_tokens, time_series_config
    )
    input_df = pd.concat([input_categorical, input_numeric], axis=1)
    output_df = pd.concat([output_categorical, output_numeric], axis=1)

    return DFComparison(input_df, output_df)


df_comp = show_results_df(
    state=state, time_series_config=time_series_config, dataset=train_ds, idx=0
)

In [ ]:
df_comp.output_df.loc[:, time_series_config.categorical_col_tokens].tail()

In [ ]:
df_comp.output_df.loc[:, time_series_config.categorical_col_tokens].tail()

In [ ]:
def plot_planets(df_pred: pd.DataFrame, df_actual: pd.DataFrame, column: str, offset=0):
    plt.figure(figsize=(15, 10))
    plt.plot(df_pred[column], label="Autogregressive")
    plt.plot(df_actual[column], label="Actual")
    plt.title(f"{column} Predictions")
    plt.legend()
    # Show ticks and grid lines every 1 step
    plt.xticks(np.arange(0, len(df_pred), 1))
    plt.grid()
    # add black line at 0 on the y axis to show the difference
    plt.axhline(0, color="black")
    plt.show()

In [ ]:
jnp.array([True, True, False, False, True]).shape

In [ ]:
x = jnp.ones((5, 20))
print(x.shape)
xx = x.at[jnp.array([True, True, False, False, True]), :].set(0)
xx

In [ ]:
@dataclass
class AutoRegressiveResults:
    numeric_inputs: jnp.array
    categorical_inputs: jnp.array

    @classmethod
    def from_ds(cls, ds: hp.TimeSeriesDS, idx: int, stop_idx: int = 10):
        inputs = ds[idx]
        numeric_inputs = inputs[0][:, :stop_idx]
        categorical_inputs = inputs[1][:, :stop_idx]
        return cls(numeric_inputs, categorical_inputs)


def auto_regressive_predictions(
    state: train_state.TrainState,
    inputs: AutoRegressiveResults,
) -> jnp.array:
    numeric_inputs = inputs.numeric_inputs
    categorical_inputs = inputs.categorical_inputs
    # get the first row that contains all nan vales
    # if nan_rows_start >= stop_idx:
    #     return inputs
    # numeric_inputs = inputs.numeric_inputs
    # categorical_inputs = inputs.categorical_inputs
    numeric_nan_columns = jnp.isnan(numeric_inputs).all(axis=1)
    categorical_nan_columns = jnp.isnan(categorical_inputs).all(axis=1)
    outputs = state.apply_fn(
        {"params": state.params},
        numeric_inputs=jnp.array([numeric_inputs]),
        categorical_inputs=jnp.array([categorical_inputs]),
        deterministic=True,
        causal_mask=False,
    )
    numeric_out = jnp.squeeze(outputs["numeric_out"])
    categorical_out = jnp.squeeze(outputs["categorical_out"])
    categorical_out = jnp.argmax(categorical_out, axis=-1)

    final_numeric_row = np.array(numeric_out[:, -1])
    final_numeric_row = final_numeric_row[:, None]  # New axis

    final_categorical_row = np.array(categorical_out[:, -1])
    final_categorical_row = final_categorical_row[:, None]  # New axis
    numeric_inputs = jnp.concatenate([numeric_inputs, final_numeric_row], axis=1)
    categorical_inputs = jnp.concatenate(
        [categorical_inputs, final_categorical_row], axis=1
    )
    numeric_inputs = numeric_inputs.at[jnp.array(numeric_nan_columns)].set(jnp.nan)
    categorical_inputs = categorical_inputs.at[jnp.array(categorical_nan_columns)].set(
        jnp.nan
    )
    inputs = (numeric_inputs, categorical_inputs)

    return inputs
    # return auto_regressive_predictions(state, inputs, stop_idx)

In [ ]:
test_inputs = AutoRegressiveResults.from_ds(train_ds, 0, 10)
# inputs_test = train_ds[0]
# test_numeric = inputs_test[0]
# test_categorical = inputs_test[1]
# print(inputs_test.shape)
for i in trange(21):
    inputs_test = auto_regressive_predictions(state, test_inputs)

# x = auto_regressive_predictions(state, test_ds[0], 10)

In [ ]:
# res = show_results_df(state, train_df, test_ds, idx=0, mask_start=30)

# plot_planets(res["pred"], res["actual_masked"], "planet2_x", offset=0)